# Testing code to automate pulling data from the ETA Site
This workbook is scratch code to develop the ETL pipeline to collect data from the ETA site. All finalized code will be implemented in polars in the ../data/ folder.

In [97]:
import polars as pl

## 1. Collect Claims Data and Convert to Months
[Unemployment Insurance Weekly Claims Data](https://oui.doleta.gov/unemploy/claims.asp): This is a state-, week-level panel of the number of new claimants of unemplyoment insurance and the number of unemployment insurance payments (weeks claimed) of active insured unemployed workers. It includes 104,304 state weeks (including Puerto Rico, District of Columbia, and the United States Virgin Islands) between August 1987 and the present.

In [98]:
# Data at https://oui.doleta.gov/unemploy/csv/ar539.csv -- timeout error for direct polars download
df = pl.read_csv("G:/My Drive/CAPP/2025_Q1_CAPP 30239/Data/ar539.csv")
df.head()

st,rptdate,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,curdate,priorwk_pub,priorwk
str,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,str,str,str,str,str
"""AK""","""2/15/1986""",6,"""2/8/1986""",2048,8,9,0,0,21230,400,133,0,0,3287,3331,0,0,16996,205263,8.28,7.53,109.96,"""B""","""12/29/1985""","""10/12/2024""","""10/7/2024""","""10/5/2024"""
"""AK""","""2/22/1986""",7,"""2/15/1986""",1635,17,8,0,0,19106,359,136,0,0,2758,2688,0,0,17355,205263,8.45,7.76,108.89,"""B""","""12/29/1985""","""10/12/2024""","""10/7/2024""","""10/5/2024"""
"""AK""","""3/1/1986""",8,"""2/22/1986""",1818,15,17,0,0,21424,380,148,0,0,3362,3324,0,0,17922,205263,8.73,7.97,109.53,"""B""","""12/29/1985""","""10/12/2024""","""10/7/2024""","""10/5/2024"""
"""AK""","""3/8/1986""",9,"""3/1/1986""",2257,20,17,0,0,21380,395,149,0,0,3273,3202,0,0,18384,205263,8.95,8.17,109.54,"""B""","""12/29/1985""","""10/12/2024""","""10/7/2024""","""10/5/2024"""
"""AK""","""3/15/1986""",10,"""3/8/1986""",2005,18,17,0,0,21812,387,145,0,0,3380,3319,0,0,18886,205263,9.2,8.32,110.57,"""B""","""12/29/1985""","""10/12/2024""","""10/7/2024""","""10/5/2024"""


In [99]:
# Rename dict for 539
rename_539 = {
    "st": "st", # State
    "rptdate": "rptdt",
    "c1": "wk_num", # Week Number
    "c2": "dt", # Reflected Week Ending
    "c3": "ct_ic", # IC - Initial Claims
    "c4": "ct_fic", # FIC - Federal Initial Claims 
    "c5": "ct_xic", # XIC - Military Initial claims
    "c6": "WSIC", # WSIC
    "c7": "WSEIC", # WSEIC
    "c8": "ct_wks", # CW - Weeks claimed
    "c9": "ct_wks_f", # FCW - Federal weeks claims
    "c10": "ct_wks_x", # XCW - Military weeks claimed
    "c11": "WSCW", # WSCW
    "c12": "WSECW", # WSECW
    "c13": "ct_wks_eb", # EBT - Total continued weeks claimed under the Federal/State Extended Benefit Program
    "c14": "ct_wks_eb_st", # EBUI - That part of EBT which represents only State UI weeks claimed under the Federal/State EB program
    "c15": "ABT",
    "c16": "ABUI",
    "c17": "ct_unemp", # AT - Moving average of unemplotyment
    "c18": "ct_emp_cv", # CE - Covered employment average over 12 months in 4 of last 6 quarter
    "c19": "rt_recip", # R - recipiency over past 13 weeks = AT / CE
    "c20": "AR", # AR - average recipiency i the past 2 years
    "c21": "P", # P = R / AR
    "c22": "ind_eb", # EB - Indicate the beginning or ending of a EB period
    "c23": "dt_eb" # Change Date - Date status changed 
}

In [100]:
# Rename and select columns
df = df.rename(rename_539)
df = df.select("st", "dt",
               "wk_num", "ct_ic", "ct_fic", "ct_xic", 
               "ct_wks", "ct_wks_f", "ct_wks_x", "ct_wks_eb",
               "rt_recip",
               "ind_eb", "dt_eb")
# TODO: Handle week end date 

In [101]:
# Extract month-level vars for collapse
# TODO: Check how this is handled by the ETA. Not in documentation here.
df = df.with_columns(
    pl.col("dt").str.to_date(format = "%-m/%-d/%Y", strict=True)
)
df = df.with_columns(
    pl.col("dt").dt.month().alias("dt_m"),
    pl.col("dt").dt.year().alias("dt_y")
)

# Collapse to month-level
# Ref: https://stackoverflow.com/a/74902919
df = df.group_by(["st", "dt_m", "dt_y"]).agg(
   pl.col(r"^(ct)_.*$").sum(),
   pl.col(r"^(rt)_.*$").mean() # Unweighted avg is fine because on time, not on pop
)
df.head()

st,dt_m,dt_y,ct_ic,ct_fic,ct_xic,ct_wks,ct_wks_f,ct_wks_x,ct_wks_eb,rt_recip
str,i8,i32,i64,i64,i64,i64,i64,i64,i64,f64
"""IA""",2,2003,13844,14,23,182334,305,539,0,2.7775
"""VT""",7,2013,2074,1,10,21023,54,62,0,1.9675
"""ID""",9,2007,6028,46,26,38965,223,269,0,1.316
"""IN""",10,2016,14348,82,52,94859,357,433,0,0.684
"""IN""",3,1995,14652,78,85,147665,986,1061,10,1.4975


## 2. Collect Demographics
[ETA 203 (Characteristics of Insured Unemployed)](https://oui.doleta.gov/unemploy/DataDownloads.asp#ETA_203): This is a state- month-level panel of the number of new recipients of unemployment insurance and their self-reported race, gender, ethnicity, and industrial characteristics. It contains 20,317 state months (including Puerto Rico, District of Columbia, and the United States Virgin Islands) between August 1994 and the present.

In [102]:
# Load csv into polars from a URL
# Link at https://oui.doleta.gov/unemploy/csv/ar203.csv -- Timeout error from ETA servers
df = pl.read_csv("G:/My Drive/CAPP/2025_Q1_CAPP 30239/Data/ar203.csv")
df.head()

st,rptdate,c1,c2,c3,c4,c12,c13,c14,c15,c16,c17,c18,c19,c20,c40,c41,c42,c43,c44,c45,c46,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56,c57,c58,c59,c60,c61,c62,c63,c64,c65,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75,c76,c77,c78,c79,c80,c81,c82,c83,c84,c85,c86,c87,c88,c89,c90,c91,c92,c93
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""AK""","""8/31/1994""","""P""",4388,4554,0,343,527,2697,2907,1635,425,288,120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""AK""","""9/30/1994""","""P""",4523,4187,0,325,541,2718,2796,1560,375,263,132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""AK""","""10/31/1994""","""P""",5456,4388,0,309,636,3109,3201,1831,406,242,110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""AK""","""11/30/1994""","""P""",7869,5094,0,451,834,4174,4196,2327,482,323,176,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""AK""","""12/31/1994""","""P""",8945,4602,0,502,838,4264,4382,2548,577,280,156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
# Create dictionary for rename
rename_203 = {
    "st": "st",  # State
    "rptdate": "dt", # Report for preiod ending
    "c2": "gen_mal",
    "c3": "gen_fem",
    "c4": "gen_na",
    "c40": "eth_hisp",
    "c41": "eth_no_hisp",
    "c42": "eth_na",
    "c43": "rac_native",
    "c44": "rac_asian",
    "c45": "rac_black",
    "c46": "rac_pac",
    "c47": "rac_white",
    "c12": "age_lt_22",
    "c13": "age_22_24",
    "c14": "age_25_34",
    "c15": "age_35_44",
    "c16": "age_45_54",
    "c17": "age_55_59",
    "c18": "age_60_64",
    "c19": "age_gte_65",
    "c20": "age_na",
}
# TODO: Add industry and occupation 

In [104]:
df = df.rename(rename_203)
# Ref: https://docs.pola.rs/user-guide/expressions/column-selections/#by-multiple-strings
df = df.select("st", "dt", pl.col(r"^(gen|eth|rac|age)_.*$"))

In [105]:
# Create month and date
df = df.with_columns(
    pl.col("dt").str.to_date(format="%-m/%-d/%Y", strict=True))
df = df.with_columns(
    pl.col("dt").dt.month().alias("dt_m"), 
    pl.col("dt").dt.year().alias("dt_y")
)
df.head()

st,dt,gen_mal,gen_fem,gen_na,age_lt_22,age_22_24,age_25_34,age_35_44,age_45_54,age_55_59,age_60_64,age_gte_65,age_na,eth_hisp,eth_no_hisp,eth_na,rac_native,rac_asian,rac_black,rac_pac,rac_white,dt_m,dt_y
str,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i8,i32
"""AK""",1994-08-31,4388,4554,0,343,527,2697,2907,1635,425,288,120,0,0,0,0,0,0,0,0,0,8,1994
"""AK""",1994-09-30,4523,4187,0,325,541,2718,2796,1560,375,263,132,0,0,0,0,0,0,0,0,0,9,1994
"""AK""",1994-10-31,5456,4388,0,309,636,3109,3201,1831,406,242,110,0,0,0,0,0,0,0,0,0,10,1994
"""AK""",1994-11-30,7869,5094,0,451,834,4174,4196,2327,482,323,176,0,0,0,0,0,0,0,0,0,11,1994
"""AK""",1994-12-31,8945,4602,0,502,838,4264,4382,2548,577,280,156,0,0,0,0,0,0,0,0,0,12,1994


## 3. CPS Cleaning
[Current Population Survey - NBER Rerelease](https://www.nber.org/research/data/current-population-survey-cps-basic-monthly-data): This is a person- / month-panel of Current Population Survey with some processing to construct relevant variables to estimate alternative unemployment rates. This data will be converted into state- / month-estimates of unemployment using various measures based on existing statistical software code I've written. I estimate that this will contain 18,309 records with the same variables of race, ethnicity, gender, and recent employment sector as ETA 203.

This gets a bit more complex due to how [unemployment rates are calculated](https://www.bls.gov/cps/definitions.htm) and [changes in the underlying survey measures](https://www.bls.gov/lau/notescps.htm). To handle this, I only work on the modern CPS implementaiton (Jan 1994-) and do not implement methodology to smooth or estimate local area unemployment statistics (LAUS). Because of this, my estimates are time bound and will not match state workforce agency recipiency releases. I can visualize this.

Note: I do not produce demographic-level unemployment statistics. The Census warns heavily against this as the CPS is approximately a 1:2500 representative survey of employment in the US. That means that for small cells, e.g., BIPOC labor in Wyoming, there might be only 1 or 2 respondents to represent a multiple thousand person population.